In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
# 1. Configuración inicial y carga de datos
os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")
nombre_archivo = "2024 10 25 - Base Original_PI.xlsx"
data = pd.read_excel(nombre_archivo)

In [16]:
# 2. Eliminar duplicados basados en el nombre del instrumento
data = data.drop_duplicates(subset='Nombre del Instrumento')

In [5]:
descripcion = data['Descripción larga'].fillna('')

In [7]:
# Convertir la columna 'Descripción larga' a texto
data['Descripción larga'] = data['Descripción larga'].astype(str)

In [8]:
import spacy

# Cargar el modelo de lenguaje en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Función para extraer palabras clave
def extract_keywords(text):
    doc = nlp(text)
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return keywords

# Aplicar la función a la columna 'Descripción larga'
data['Palabras Clave'] = data['Descripción larga'].apply(extract_keywords)

# Mostrar las primeras filas con las palabras clave extraídas
data[['Descripción larga', 'Palabras Clave']].head()

,Descripción larga,Palabras Clave
0,A partir del modelo pedagógico de la entidad p...,"[modelo, pedagógico, entidad, ciclo, vida, des..."
1,Los requisitos para vincularse a Hoy y Mañana ...,"[requisitos, vincularse, Mañana, BEPS, ciudada..."
2,La feria Silverexpo se realiza en alianza con ...,"[feria, Silverexpo, realiza, alianza, Corferia..."
3,La adecuación de tierras es el servicio públic...,"[adecuación, tierras, servicio, público, contr..."
4,MEJORAR LA TRANSFERENCIA Y APROPIACIÓN DEL CON...,"[MEJORAR, TRANSFERENCIA, APROPIACIÓN, CONOCIMI..."


In [11]:
# Vectorizar las palabras clave usando TF-IDF
vectorizer_keywords = TfidfVectorizer()
tfidf_matrix_keywords = vectorizer_keywords.fit_transform(data['Palabras Clave'].apply(lambda x: ' '.join(x)))

# Calcular la similitud de coseno entre las palabras clave
keywords_sim = cosine_similarity(tfidf_matrix_keywords) * 100

# Encontrar y etiquetar las parejas de instrumentos que se parecen
similar_pairs = []
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        similarity = keywords_sim[i, j]
        if similarity > 80:  # Considerar solo similitudes mayores a 0
            similar_pairs.append((data.iloc[i]['Nombre del Instrumento'], data.iloc[j]['Nombre del Instrumento'], similarity))

# Imprimir las parejas de instrumentos que se parecen
for pair in similar_pairs:
    print(f"Instrumento 1: {pair[0]}, Instrumento 2: {pair[1]}, Similitud: {pair[2]:.2f}%")

Instrumento 1: Educación Financiera, Instrumento 2: Talleres de educación financiera, Similitud: 86.21%
Instrumento 1: Becas en el Exterior para estudiantes y profesionales colombianos, Instrumento 2: Becas en el Exterior para estudiantes y profesionales colombianos 2024-2, Similitud: 97.42%
Instrumento 1: Becas en el Exterior para estudiantes y profesionales colombianos 2024-2, Instrumento 2: Becas en el exterior para estudiantes y profesionales Colombianos 2025-1, Similitud: 80.22%
Instrumento 1: Crédito educativo a traves de la administración de recursos de terceros, Instrumento 2: Crédito educativo a través de la administración de recursos de terceros, Similitud: 81.46%
Instrumento 1: Colecciones biológicas 2025, Instrumento 2: Servicio de modelamiento para la conservación de la biodiversidad 2025, Similitud: 86.24%
Instrumento 1: Beneficios Tributarios por la Vinculación de Doctores a las empresas, Instrumento 2: REGISTRO DE PROPUESTAS QUE ACCEDERÁN A LOS BENEFICIOS TRIBUTARIOS PO

In [19]:
# 3. Definir las columnas para cada categoría
usuarios_cols = ['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 
                'Entidades de soporte', 'Personas naturales']
objetivo_cols = ['Innovación', 'Emprendimiento', 'Economía Popular', 
                'Transferencia de Conocimiento y Tecnología', 'Investigación', 
                'Formación de Capital Humano', 'Propiedad intelectual', 'Calidad', 
                'Clúster o encadenamientos', 'Financiación', 'Comercialización', 
                'Formalización', 'Crecimiento Sostenible', 'Inclusión Financiera', 
                'Comercio Electrónico', 'Educación económica y financiera', 
                'Competencias Digitales']
apoyo_cols = ['Formación de Talento Humano', 'Tipo de Sistemas de Información', 
              'Asistencia técnica y Consultoría', 'Tipo de Redes de Colaboración', 
              'Apoyo Financiero', 'Incentivos Tributarios', 'Eventos', 'Compra Pública', 
              'Bonos o Vouchers', 'Premios y Reconocimientos', 'Instrumentos Regulatorios']



In [7]:
# 4. Preparar y normalizar matrices
scaler = StandardScaler()

# Preparar matrices binarias y normalizar
def prepare_normalized_matrix(df, columns):
    binary_matrix = df[columns].replace({"Si": 1, "No": 0}).fillna(0).values
    return scaler.fit_transform(binary_matrix)

usuarios_matrix = prepare_normalized_matrix(data, usuarios_cols)
objetivo_matrix = prepare_normalized_matrix(data, objetivo_cols)
apoyo_matrix = prepare_normalized_matrix(data, apoyo_cols)



KeyError: "['Academia_', 'Personas Naturales'] not in index"

In [18]:
# 5. Calcular similitudes usando coseno
usuarios_sim = cosine_similarity(usuarios_matrix) * 100
objetivo_sim = cosine_similarity(objetivo_matrix) * 100
apoyo_sim = cosine_similarity(apoyo_matrix) * 100

In [31]:
# 7.1. Combinar análisis de similitud por coseno con hallazgos de Palabras Clave para todos los instrumentos
combined_results = []

# Iterar sobre todos los pares posibles de instrumentos
for i in range(len(data)):
    for j in range(i + 1, len(data)):  # Evitar repetir pares ya evaluados
        instrumento1 = data['Nombre del Instrumento'].iloc[i]
        instrumento2 = data['Nombre del Instrumento'].iloc[j]

        # Obtener similitudes
        similitudes = {
            'USUARIOS': usuarios_sim[i, j],
            'OBJETIVO': objetivo_sim[i, j],
            'APOYO': apoyo_sim[i, j],
            #'ENTIDAD': entidad_sim[i, j],
            'TEXTO': keywords_sim[i, j]
        }

        # Determinar nivel de articulación según el diagrama
        if all([similitudes['USUARIOS'] >= 80, similitudes['OBJETIVO'] >= 80, similitudes['APOYO'] >= 80]):
            nivel_articulacion = "Fusionar"
        elif all([similitudes['USUARIOS'] >= 80, similitudes['OBJETIVO'] >= 80, similitudes['APOYO'] < 80]):
            nivel_articulacion = "Definir roles"
        elif all([similitudes['USUARIOS'] >= 80, similitudes['OBJETIVO'] >= 80]) and similitudes['APOYO'] < 80:
            nivel_articulacion = "Empaquetar"
        elif similitudes['USUARIOS'] >= 80 and similitudes['OBJETIVO'] < 80 and similitudes['APOYO'] < 80:
            nivel_articulacion = "Generar rutas"
        else:
            continue  # No cumple con los criterios, omitir

        # Guardar resultados
        combined_results.append({
            'Instrumento1': instrumento1,
            'Instrumento2': instrumento2,
            'Similitud Palabras Clave': similitudes['TEXTO'],
            'Similitud Usuarios': similitudes['USUARIOS'],
            'Similitud Objetivo': similitudes['OBJETIVO'],
            'Similitud Apoyo': similitudes['APOYO'],
            #'Similitud Entidad': similitudes['ENTIDAD'],
            'Nivel de Articulación': nivel_articulacion
        })

# Crear DataFrame con resultados combinados
combined_results_df = pd.DataFrame(combined_results)
combined_results_df.head()

# Guardar los resultados en un archivo Excel
combined_results_df.to_excel("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/Informes por sectores/Resultados/Resultados_Combinados_Todos.xlsx", index=False)


In [30]:
combined_results_df.to_excel("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/Informes por sectores/Resultados/Resultados_Combinados.xlsx", index=False)

In [12]:
# 3. Definir las columnas para cada categoría
usuarios_cols = ['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 
                'Entidades de soporte', 'Personas naturales']
objetivo_cols = ['Innovación', 'Emprendimiento', 'Economía Popular', 
                'Transferencia de Conocimiento y Tecnología', 'Investigación', 
                'Formación de Capital Humano', 'Propiedad intelectual', 'Calidad', 
                'Clúster o encadenamientos', 'Financiación', 'Comercialización', 
                'Formalización', 'Crecimiento Sostenible', 'Inclusión Financiera', 
                'Comercio Electrónico', 'Educación económica y financiera', 
                'Competencias Digitales']
apoyo_cols = ['Formación de Talento Humano', 'Tipo de Sistemas de Información', 
              'Asistencia técnica y Consultoría', 'Tipo de Redes de Colaboración', 
              'Apoyo Financiero', 'Incentivos Tributarios', 'Eventos', 'Compra Pública', 
              'Bonos o Vouchers', 'Premios y Reconocimientos', 'Instrumentos Regulatorios']

In [21]:
# Criterios ArCo PI

usuarios_cols = ["Microempresa", "Pequeña Empresa", 'Mediana Empresa', 'Gobierno ', 'Academia', 'Personas naturales']


objetivo_cols = ["Transferencia de Conocimiento y Tecnología",
    "Propiedad intelectual","Si el instrumento es de propiedad intelectual por favor marque aquí!_y",
    "¿Qué tipos de derechos de propiedad intelectual contempla?"]


apoyo_cols = ["Formación de Talento Humano", "Tipo de Sistemas de Información",
    "Asistencia técnica y Consultoría", "Tipo de Redes de Colaboración", "Apoyo Financiero",
    "Incentivos Tributarios", "Eventos", "Compra Pública", "Bonos o Vouchers", "Premios y Reconocimientos",
    "Instrumentos Regulatorios"]

In [22]:
# 4. Preparar y normalizar matrices
scaler = StandardScaler()

# Preparar matrices binarias y normalizar
def prepare_normalized_matrix(df, columns):
    binary_matrix = df[columns].replace({"Si": 1, "No": 0}).fillna(0).values
    return scaler.fit_transform(binary_matrix)

usuarios_matrix = prepare_normalized_matrix(data, usuarios_cols)
objetivo_matrix = prepare_normalized_matrix(data, objetivo_cols)
apoyo_matrix = prepare_normalized_matrix(data, apoyo_cols)

In [23]:
# 5. Calcular similitudes usando coseno
usuarios_sim = cosine_similarity(usuarios_matrix) * 100
objetivo_sim = cosine_similarity(objetivo_matrix) * 100
apoyo_sim = cosine_similarity(apoyo_matrix) * 100


In [25]:
# 6. Calcular similitud textual usando TF-IDF
# Combinar descripción y criterios en un solo texto
data['texto_combinado'] = data['Descripción'].fillna('') + ' ' + \
                         data[usuarios_cols + objetivo_cols + apoyo_cols].apply(
                             lambda x: ' '.join([col for col, val in x.items() if val == 'Si']), 
                             axis=1
                         )

vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(data['texto_combinado'])
texto_sim = cosine_similarity(tfidf_matrix) * 100

In [26]:
# 7. Crear matriz de similitud para entidades
n = len(data)
entidades = data['Entidad'].values
entidad_sim = np.equal.outer(entidades, entidades).astype(float) * 100

In [27]:
# 8. Definir umbral de similitud
UMBRAL = 70

In [23]:
# 9. Analizar cada instrumento y crear resultados
results = []
for i in range(n):
    # Calcular similitud total promedio
    total_sim = (usuarios_sim[i] + objetivo_sim[i] + apoyo_sim[i] + 
                entidad_sim[i] + texto_sim[i]) / 5
    total_sim[i] = -1  # Excluir comparación consigo mismo
    
    # Encontrar el mejor match
    best_match_idx = np.argmax(total_sim)
    
    # Obtener similitudes con el mejor match
    similitudes = {
        'USUARIOS': usuarios_sim[i, best_match_idx],
        'OBJETIVO': objetivo_sim[i, best_match_idx],
        'APOYO': apoyo_sim[i, best_match_idx],
        'ENTIDAD': entidad_sim[i, best_match_idx],
        'TEXTO': texto_sim[i, best_match_idx]
    }
    
    # Determinar similitudes altas/bajas según el umbral
    similar_usuarios = similitudes['USUARIOS'] >= UMBRAL
    similar_objetivo = similitudes['OBJETIVO'] >= UMBRAL
    similar_apoyo = similitudes['APOYO'] >= UMBRAL
    similar_entidad = similitudes['ENTIDAD'] >= UMBRAL
    
    # Determinar nivel de articulación según el diagrama
    if all([similar_usuarios, similar_objetivo, similar_apoyo, similar_entidad]):
        nivel_articulacion = "2. Fusionar"
    elif all([similar_usuarios, similar_objetivo, similar_apoyo]) and not similar_entidad:
        nivel_articulacion = "3. Definir roles"
    elif all([similar_usuarios, similar_objetivo]) and not similar_apoyo:
        nivel_articulacion = "4. Empaquetar"
    elif similar_usuarios and not similar_objetivo:
        nivel_articulacion = "5. Generar rutas"
    else:
        nivel_articulacion = "1. Renunciar"
    
    # Guardar resultados
    results.append({
        'Instrumento1': data.iloc[i]['Nombre del Instrumento'],
        'Descripción1': data.iloc[i]['Descripción'],
        'Entidad1': data.iloc[i]['Entidad'],
        'Instrumento2': data.iloc[best_match_idx]['Nombre del Instrumento'],
        'Descripción2': data.iloc[best_match_idx]['Descripción'],
        'Entidad2': data.iloc[best_match_idx]['Entidad'],
        'Nivel de Articulación': nivel_articulacion,
        'Similitud Total': total_sim[best_match_idx],
        'Similitud Usuarios': similitudes['USUARIOS'],
        'Similitud Objetivo': similitudes['OBJETIVO'],
        'Similitud Apoyo': similitudes['APOYO'],
        'Similitud Entidad': similitudes['ENTIDAD'],
        'Similitud Texto': similitudes['TEXTO']
    })

In [24]:
# 10. Crear DataFrame con resultados y exportar
results_df = pd.DataFrame(results)
output_filename = f"Analisis_Articulacion_Instrumentos_{datetime.now().strftime('%Y%m%d')}.xlsx"
results_df.to_excel(output_filename, index=False)

print(f"Análisis completado. Los resultados se han guardado en '{output_filename}'.")

Análisis completado. Los resultados se han guardado en 'Analisis_Articulacion_Instrumentos_20241018.xlsx'.
